In [1]:
import tensorflow as tf
import requests
import base64
import json

2024-08-01 15:55:34.237461: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-01 15:55:34.330496: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-01 15:55:34.511049: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-01 15:55:34.511196: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-01 15:55:34.546766: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [9]:
def prepare_json(features):
    # Extract individual feature values from the input list
    pregnancies, glucose, blood_pressure, skin_thickness, insulin, bmi, pedigree, age = features
    
    # Create a feature specification with individual features
    feature_spec = {
        "Pregnancies": tf.train.Feature(int64_list=tf.train.Int64List(value=[pregnancies])),
        "Glucose": tf.train.Feature(int64_list=tf.train.Int64List(value=[glucose])),
        "BloodPressure": tf.train.Feature(int64_list=tf.train.Int64List(value=[blood_pressure])),
        "SkinThickness": tf.train.Feature(int64_list=tf.train.Int64List(value=[skin_thickness])),
        "Insulin": tf.train.Feature(int64_list=tf.train.Int64List(value=[insulin])),
        "BMI": tf.train.Feature(float_list=tf.train.FloatList(value=[bmi])),
        "DiabetesPedigreeFunction": tf.train.Feature(float_list=tf.train.FloatList(value=[pedigree])),
        "Age": tf.train.Feature(int64_list=tf.train.Int64List(value=[age]))
    }
    
    # Create an Example protocol buffer
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    # Encode the Example protocol buffer in base64
    result = [{'examples': {'b64': base64.b64encode(example).decode()}}]
    
    # Return the JSON formatted payload for the model
    return json.dumps(
        {
            "signature_name": "serving_default",
            "instances": result
        }
    )

# Prepare the input numerical features for the model prediction
input_features = [6, 148, 72, 35, 0, 33.6, 0.527, 50]  # Example input values

# Make a POST request to the model server with the prepared JSON
response = requests.post(
    "https://diabetes-detection-8a31c5bf0c49.herokuapp.com/v1/models/serving_model:predict", 
    data=prepare_json(input_features),
    headers={"Content-Type": "application/json"}
).json()

print(response)

{'predictions': [[0.999954641]]}
